In [1]:
import tensorflow
print(tensorflow.__version__)

2.11.0


In [2]:
!pip install tensorflow==1.13.1 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.6/367.6 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.11.0
    Uninstalling tensorflow-estimator-2.11.0:
      Successfully uninstalled tensorflow-estimator-2.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.11.2
    Uninstalling tensorboard-2.11.2:
      Successfully uninstalled tensorboard-2.11.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.11.0
    Uninstalling tensorflow-2.11.0:
      Successfully uninstalled tensorflow-2.11.0
ERROR: pip's dependency re

In [24]:
#####UTILS.Py####
import os
import time
import numpy as np


def SetDefaultAsNatural(FLAGS):
    if hasattr(FLAGS, 'epoch') and FLAGS.epoch < 0:
        FLAGS.epoch = 60

    if hasattr(FLAGS, 'learning_rate') and FLAGS.learning_rate < 0:
        FLAGS.learning_rate = 0.1

    if hasattr(FLAGS, 'regularizer') and FLAGS.regularizer < 0:
        FLAGS.regularizer = 1e-4

    if hasattr(FLAGS, 'alpha') and FLAGS.alpha < 0:
        FLAGS.alpha = 1.0 / 3

def PrintResults(alpha, results):

    result_text = "%.2f" % alpha
    for i in range(int(len(results)/2)):
        result_text += ", train-idx%d: %.4f" % (i+1, results[i])
    for i in range(int(len(results)/2)):
        result_text += ", test-idx%d: %.4f" % (i+1, results[i + int(len(results)/2)])
    print(result_text)

In [20]:
import tensorflow as tf
import numpy as np
import math

class Linear(object):
    def __init__(self, h_in, out_len, trainable=True):
        in_len = np.prod(h_in.get_shape().as_list()[1:])
        initial = tf.truncated_normal([in_len, out_len], stddev = 0.1)
        self.W = tf.Variable(initial, trainable=trainable)
        self.b = tf.Variable(tf.constant(0.1, shape=[out_len]), trainable=trainable)
        self.h_out = tf.matmul(tf.reshape(h_in, [-1,in_len]), self.W) + self.b


class RegLinear(Linear):
    def __init__(self, Linear, FMo=None, trainable=False):
        shape = Linear.W.get_shape().as_list()
        self.W = tf.Variable(tf.constant(0.0, shape=shape), trainable=trainable)
        self.b = tf.Variable(tf.constant(0.0, shape=[shape[-1]]), trainable=trainable)

        if FMo == None:
            self.reg_obj = tf.reduce_sum(tf.square(Linear.W - self.W))
            self.reg_obj += tf.reduce_sum(tf.square(Linear.b - self.b)) 
        else:
            self.reg_obj = tf.reduce_sum(tf.mul(FMo.W,tf.square(Linear.W - self.W)))
            self.reg_obj += tf.reduce_sum(tf.mul(FMo.b,tf.square(Linear.b - self.b)))


class DropLinear():
    def __init__(self, h_in, out_len, keep_prob, trainable=True):
        in_len = np.prod(h_in.get_shape().as_list()[1:])
        initial = tf.truncated_normal([in_len, out_len], stddev = 0.1)
        self.W = tf.Variable(initial,trainable=trainable)
        self.b = tf.Variable(tf.constant(0.1,shape=[out_len]),trainable=trainable)
        
        self.dropbase = RegLinear(self)

        self.h_out = tf.nn.dropout(tf.matmul(h_in, self.W) + self.b, keep_prob)\
            + tf.matmul(h_in, self.dropbase.W) + self.dropbase.b

In [14]:
import os
import time
import numpy as np

import tensorflow as tf

def CopyLayers(sess, Lsc, Ltr = None, trainable_info=None):
    if trainable_info == None:
        trainable_info = np.tile([False], len(Lsc))

    for l in range(len(Lsc)):
        if len(Ltr) <= l:
            Ltr.append(RegLinear(Lsc[l], trainable=trainable_info[l]))
        sess.run(Ltr[l].W.assign(Lsc[l].W.eval(sess)))
        sess.run(Ltr[l].b.assign(Lsc[l].b.eval(sess)))

    return Ltr

def CopyLayerValues(sess, Lsc, Ltr = None):
    if Ltr == None: Ltr = []
    for l in range(len(Lsc)):
        if len(Ltr) < l + 1: Ltr.append({})
        Ltr[l]['W'] = Lsc[l].W.eval(sess)
        Ltr[l]['b'] = Lsc[l].b.eval(sess)
    return Ltr

def ZeroLayers(sess, L):
    for l in range(len(L)):
        shape = L[l].W.get_shape().as_list()
        sess.run(L[l].W.assign(np.zeros(shape)))
        sess.run(L[l].b.assign(np.zeros(shape[-1])))

def AddLayers(sess, L1, L2, Ltr):
    # TODO: ad-hoc, it should be resolved in a natural way.
    op = []
    for l in range(len(L1)):
        v = sess.run([L1[l].W, L2[l].W, L1[l].b, L2[l].b])
        op += [Ltr[l].W.assign(v[0]+v[1]), Ltr[l].b.assign(v[2]+v[3])]
    sess.run(op)

def AddMultiTaskLayers(sess, Ls, Ltr, Lw, noOfTask):
    if len(Lw) < 1 or len(Ls) < 1:
        return

    ops = []
    noOfLayer = len(Lw[0])

    for i in range(noOfLayer):
        val_W = 0
        val_b = 0
        for j in range(noOfTask):
            val_W += Ls[j][i]['W'] * Lw[j][i]['W']
            val_b += Ls[j][i]['b'] * Lw[j][i]['b']

        ops.append(Ltr[i].W.assign(val_W))
        ops.append(Ltr[i].b.assign(val_b))

    sess.run(ops)

def Lw_maker(sess, L, alpha_info, adhoc_mnist=False):
    Layers_out = [];
    op = [];
    for l in range(len(L)):
        shape = L[l].W.get_shape().as_list()
        rl = RegLinear(L[l])
        op += [rl.W.assign(np.zeros(shape)+alpha_info[l])]
        op += [rl.b.assign(np.zeros([shape[1],])+alpha_info[l])]
        Layers_out.append(rl)
    if adhoc_mnist: adhoc_MNIST_(sess,Layers_out)
    sess.run(op)
    return Layers_out

def CalculateWeighingBase(sess, L1, L2, alpha):
    Lw = CopyLayers(sess, L1, [])
    op = []
    for l in range(len(L1)):
        (W1, W2, b1, b2) = sess.run([L1[l].W, L2[l].W, L1[l].b, L2[l].b])
        op += [Lw[l].W.assign( alpha*W2 / ((1-alpha)*W1 + alpha*W2)) ]
        op += [Lw[l].b.assign( alpha*b2 / ((1-alpha)*b1 + alpha*b2)) ]
    sess.run(op)
    return Lw

def UpdateMultiTaskLwWithAlphas(L, alpha_list, noOfTask):
    Lw = []

    for i in range(noOfTask):
        if len(Lw) < i + 1:
            Lw.append([])
        for l in range(len(L)):
            if len(Lw[i]) < l + 1:
                Lw[i].append({})
            Lw[i][l]['W'] = np.zeros(L[l]['W'].shape) + alpha_list[i]
            Lw[i][l]['b'] = np.zeros(L[l]['b'].shape) + alpha_list[i]
    return Lw

def UpdateMultiTaskWeightWithAlphas(Fs, alpha_list, noOfTask):
    Lw = []

    total_W = 0
    total_b = 0
    noOfLayer = len(Fs[0])

    # total: denominator
    total_W = []
    total_b = []
    for i in range(noOfLayer):
        total_W.append(alpha_list[noOfTask -1] * Fs[noOfTask - 1][i]['W'])
        total_b.append(alpha_list[noOfTask -1] * Fs[noOfTask - 1][i]['b'])
        for j in range(noOfTask - 1):
            total_W[i] += alpha_list[j] * Fs[j][i]['W']
            total_b[i] += alpha_list[j] * Fs[j][i]['b']

    # calculating layer weight
    for i in range(noOfTask):
        if len(Lw) < i + 1:
            Lw.append([])
        for j in range(noOfLayer):
            if len(Lw[i]) < j + 1:
                Lw[i].append({})
            val_W = alpha_list[i] * Fs[i][j]['W'] / total_W[j]
            val_b = alpha_list[i] * Fs[i][j]['b'] / total_b[j]
            Lw[i][j]['W'] = val_W
            Lw[i][j]['b'] = val_b

    return Lw

def PrintLayers(sess, L):
    print("Welcome to my PrintLayers.")
    for l in range(len(L)):
        shape = np.asarray(L[l].W.get_shape().as_list())
        (W_, b_) = sess.run([L[l].W, L[l].b])
        print("Level", l+1, ":", np.r_[shape, W_[0,0:2],W_[-1,-2:],b_[0],b_[-1]])
    print("")

def PrintLayers2(L):
    print("Welcome to my PrintLayers.")
    for l in range(len(L)):
        shape = L[l]['W'].shape
        W_, b_ = [L[l]['W'], L[l]['b']]
        print("Level", l+1, ":", np.r_[shape, W_[0,0:2],W_[-1,-2:],b_[0],b_[-1]])
    print("")

In [15]:
###MNIST.py####
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

# import utils
# from model import imm


def XycPackage():
    """
    Load Dataset and set package.
    """
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

    noOfTask = 3
    x = []
    x_ = []
    y = []
    y_ = []
    xyc_info = []

    x.append(np.concatenate((mnist.train.images,mnist.validation.images)))
    y.append(np.concatenate((mnist.train.labels,mnist.validation.labels)))
    x_.append(mnist.test.images)
    y_.append(mnist.test.labels)
    xyc_info.append([x[0], y[0], 'train-idx1'])

    for i in range(1, noOfTask):
        idx = np.arange(784)                 # indices of shuffling image
        np.random.shuffle(idx)
        
        x.append(x[0].copy())
        x_.append(x_[0].copy())
        y.append(y[0].copy())
        y_.append(y_[0].copy())

        x[i] = x[i][:,idx]           # applying to shuffle
        x_[i] = x_[i][:,idx]

        xyc_info.append([x[i], y[i], 'train-idx%d' % (i+1)])

    for i in range(noOfTask):
        xyc_info.append([x_[i], y_[i], 'test-idx%d' % (i+1)])

    return x, y, x_, y_, xyc_info

In [21]:
import tensorflow as tf
import numpy as np
import math

# from Linear import Linear, RegLinear, DropLinear


#########################################################################################

class TransferNN(object):
    def __init__(self, node_info, optim=('Adam',1e-4), name='[tf/NN]', trainable_info=None, keep_prob_info=None):

        self.name = name
        self.optim = optim
        self.node_info = node_info

        if keep_prob_info == None:
            keep_prob_info = [0.8] + [0.5] * (len(node_info) - 2)
        self.keep_prob_info = np.array(keep_prob_info)
        self.eval_keep_prob_info = np.array([1.0] * (len(node_info) - 1))

        if trainable_info == None:
            trainable_info = [True] * len(node_info)
        self.trainable_info = trainable_info

        self.x = tf.placeholder(tf.float32, shape=[None, np.prod(node_info[0])])
        self.y_ = tf.placeholder(tf.float32, shape=[None, node_info[-1]])
        self.drop_rate = tf.placeholder(tf.float32, shape=[len(node_info) - 1])

        self._BuildModel()
        self._CrossEntropyPackage(optim)


    def _BuildModel(self):
        h_out_prev = tf.nn.dropout(self.x, self.drop_rate[0])

        self.Layers = []
        self.Layers_dropbase = []
        for l in range(1, len(self.node_info)-1):
            self.Layers.append(DropLinear(h_out_prev, self.node_info[l], self.drop_rate[l]))
            self.Layers_dropbase.append(self.Layers[-1].dropbase)
            
            h_out_prev = tf.nn.relu(self.Layers[-1].h_out)

        self.Layers.append(DropLinear(h_out_prev, self.node_info[-1], 1.0))
        self.Layers_dropbase.append(self.Layers[-1].dropbase)
        self.y = self.Layers[-1].h_out

    def _OptimizerPackage(self, obj, optim):
        if optim[0] == 'Adam': return tf.train.AdamOptimizer(optim[1]).minimize(obj)
        elif optim[0] == 'SGD': return tf.train.GradientDescentOptimizer(optim[1]).minimize(obj)
        elif optim[0] == 'Momentum': return tf.train.MomentumOptimizer(optim[1][0],optim[1][1]).minimize(obj)

    def _CrossEntropyPackage(self, optim):
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.y))
        self.train_step = self._OptimizerPackage(self.cross_entropy, optim) 
        self.correct_prediction = tf.equal(tf.argmax(self.y,1), tf.argmax(self.y_,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))


    def RegPatch(self, delta):
        self.reg_obj = 0
        self.Layers_reg = []

        for l in range(0,len(self.Layers)):
            self.Layers_reg.append(RegLinear(self.Layers[l]))
            self.reg_obj += delta * self.Layers_reg[l].reg_obj

        cel = tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.y)
        self.cross_entropy = tf.reduce_mean(cel) + self.reg_obj
        self.train_step = self._OptimizerPackage(self.cross_entropy, self.optim)

    def CalculateFisherMatrix(self, sess, x, y, mb=1000):
        FM = []
        data_size = x.shape[0]
        total_step = int(math.ceil(float(data_size)/mb))

        for step in range(total_step):
            ist = (step * mb) % data_size
            ied = min(ist + mb, data_size)
            y_sample = tf.reshape(tf.one_hot(tf.multinomial(self.y, 1), 10), [-1, 10])
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_sample, logits=self.y))
            for l in range(len(self.Layers)):
                if len(FM) < l + 1:
                    FM.append({})
                    FM[l]['W'] = np.zeros(self.Layers[l].W.get_shape().as_list())
                    FM[l]['b'] = np.zeros(self.Layers[l].b.get_shape().as_list())
                W_grad = tf.reduce_sum(tf.square(tf.gradients(cross_entropy,[self.Layers[l].W])), 0)
                b_grad = tf.reduce_sum(tf.square(tf.gradients(cross_entropy,[self.Layers[l].b])), 0)
                W_grad_val, b_grad_val = sess.run([W_grad, b_grad],
                                    feed_dict={ self.x:x[ist:ied],
                                                self.drop_rate:self.eval_keep_prob_info})
                FM[l]['W'] += W_grad_val
                FM[l]['b'] += b_grad_val

        for l in range(len(self.Layers)):
            FM[l]['W'] += 1e-8
            FM[l]['b'] += 1e-8
        
        return FM

    def Train(self, sess, x, y, x_, y_, epoch, mb=50):
        data_size = x.shape[0]
        total_step = int(math.ceil(float(data_size)/mb))

        for e in range(epoch):
            train_acc = 0
            for step in range(total_step):
                ist = (step * mb) % data_size
                ied = min(ist + mb, data_size)

                _, acc = sess.run([self.train_step, self.accuracy], 
                                    feed_dict={ self.x:x[ist:ied],
                                                self.y_:y[ist:ied],
                                                self.drop_rate:self.keep_prob_info})
                train_acc += (ied - ist) * acc
            train_acc /= data_size

            test_acc = self.Test(sess, [[x_,y_,""]], 1000, False)[0]
            print("(%d, %d, %d, %.4f, %.4f)" % (e+1, (e+1)*total_step,
                (e+1)*data_size, train_acc, test_acc))

    def Test(self, sess, xyc_info, mb=1000, debug=True): #ti: triple_info
        acc_ret = []

        for l in range(len(xyc_info)):
            x_, y_, c = xyc_info[l]
            comment = self.name + c
            acc = self._Test(sess, x_, y_, mb)
            acc_ret.append(acc)

            if debug: 
                print("%s accuracy : %.4f" % (comment, acc))

        return acc_ret

    def _Test(self, sess, x_, y_, mb):
        acc = 0
        data_size = x_.shape[0]
        for step in range(int(math.ceil(float(data_size)/mb))):
            ist = (step * mb) % data_size
            ied = min(ist + mb, data_size)
            acc += (ied - ist) * sess.run(self.accuracy, 
                                feed_dict={ self.x:x_[ist:ied], 
                                            self.y_:y_[ist:ied], 
                                            self.drop_rate:self.eval_keep_prob_info})
        acc /= data_size
        return acc


    def TestTasks(self, sess, x, y, x_, y_, mb=1000, debug=True): #ti: triple_info
        """
        test tasks using x, y, x_, y_ data.
        Args:
            x: list of original and shuffled input training data
            y: label of training image
            x_: list of original and shuffled input test data
                (the size should be same with the size of x)
            y_: label of test image
        Returns:
            ret: list of accuracy
                [training_accuracies, ..., test_accuracies, ...]
        """
        xyc_info = []
        for i in range(len(x)):
            xyc_info.append([x[i], y[i], 'train-idx%d' % i])
        for i in range(len(x_)):
            xyc_info.append([x_[i], y_[i], 'test-idx%d' % i])
     
        return self.Test(sess, xyc_info, mb=mb, debug=debug)

    def TestAllTasks(self, sess, x_tasks, y_tasks, mb=1000, debug=True): #ti: triple_info
        acc_ret = []
        for l in range(len(x_tasks)):
            x_ = x_tasks[l]
            y_ = y_tasks[l]
            acc = self._Test(sess, x_, y_, mb)
            acc_ret.append(acc)
        print(acc_ret)
        if debug: 
            print("%s all test accuracy : %.4f" % (self.name, np.average(acc_ret)))

        return acc_ret

In [25]:
import time
import argparse
import numpy as np
import tensorflow as tf


mean_imm = True
mode_imm = False
alpha = 0.8
optimizer = 'SGD'
learning_rate = 1e-3
epoch = 3
batch_size = 50

no_of_task = 3
no_of_node = [784,800,800,10]
keep_prob_info = [0.8, 0.5, 0.5]


# data preprocessing
x, y, x_, y_, xyc_info = XycPackage()

start = time.time()

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    mlp = TransferNN(no_of_node, (optimizer, learning_rate), keep_prob_info=keep_prob_info)

    sess.run(tf.global_variables_initializer())

    L_copy = []
    FM = []
    for i in range(no_of_task):
        print("")
        print("================= Train task #%d (%s) ================" % (i+1, optimizer))

        mlp.Train(sess, x[i], y[i], x_[i], y_[i], epoch, mb=batch_size)
        mlp.Test(sess, [[x[i],y[i]," train"], [x_[i],y_[i]," test"]])

        if mean_imm or mode_imm:
            L_copy.append(CopyLayerValues(sess, mlp.Layers))
        if mode_imm:
            FM.append(CalculateFisherMatrix(sess, x[i], y[i]))

    mlp.TestAllTasks(sess, x_, y_)


    alpha_list = [(1-alpha)/(no_of_task-1)] * (no_of_task-1)
    alpha_list.append(alpha)
    ######################### Mean-IMM ##########################
    if mean_imm:
        print("")
        print("Main experiment on %s + Mean-IMM, shuffled MNIST" % optimizer)
        print("============== Train task #%d (Mean-IMM) ==============" % no_of_task)

        LW = UpdateMultiTaskLwWithAlphas(L_copy[0], alpha_list, no_of_task)
        AddMultiTaskLayers(sess, L_copy, mlp.Layers, LW, no_of_task)
        ret = mlp.TestTasks(sess, x, y, x_, y_, debug = False)
        PrintResults(alpha, ret)

        mlp.TestAllTasks(sess, x_, y_)

    ######################### Mode-IMM ##########################
    if mode_imm:
        print("")
        print("Main experiment on %s + Mode-IMM, shuffled MNIST" % optimizer)
        print("============== Train task #%d (Mode-IMM) ==============" % no_of_task)

        LW = UpdateMultiTaskWeightWithAlphas(FM, alpha_list, no_of_task)
        AddMultiTaskLayers(sess, L_copy, mlp.Layers, LW, no_of_task)
        ret = mlp.TestTasks(sess, x, y, x_, y_, debug = False)
        PrintResults(alpha, ret)

        mlp.TestAllTasks(sess, x_, y_)

    print("")
    print("Time: %.4f s" % (time.time()-start))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

================= Train task #1 (SGD) ================
(1, 1200, 60000, 0.0986, 0.0980)
(2, 2400, 120000, 0.0987, 0.0980)
[tf/NN] train accuracy : 0.0987
[tf/NN] test accuracy : 0.0980

================= Train task #2 (SGD) ================
(1, 1200, 60000, 0.0987, 0.0980)
(2, 2400, 120000, 0.0987, 0.0980)
[tf/NN] train accuracy : 0.0987
[tf/NN] test accuracy : 0.0980

================= Train task #3 (SGD) ================
(1, 1200, 60000, 0.0987, 0.0980)
(2, 2400, 120000, 0.0987, 0.0980)
[tf/NN] train accuracy : 0.0987
[tf/NN] test accuracy : 0.0980
[0.09800000116229057, 0.09800000116229057, 0.09800000116229057]
[tf/NN] all test accuracy : 0.0980

Main experiment on SGD + Mean-IMM, shuffled MNIST
============== Train task #3 (Mean-IMM) ==============
-1.00, train-idx1: 0.0987, train-idx2: 0.09